In [16]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score,confusion_matrix

# url = "https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv"
df = pd.read_csv('sms.tsv', sep='\t', header=None, names=['l', 'message'])
df['message'] = df['message'].fillna('')  # Handle NaN values

x=df['message']
y=df['l']

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=23)

vectorizer=CountVectorizer()
x_v_train=vectorizer.fit_transform(x_train)
x_v_test=vectorizer.transform(x_test)

y_train=y_train.values
y_test=y_test.values

class mulitnomialNB():
        def fit(self,x,y):
                self.classes=np.unique(y)
                self.class_log_prior={}
                self.feature_log_prior={}
                self.alpha=1

                total_docs=len(y)
                for c in self.classes:
                        x_c=x[y==c]
                        total_c_docs=x_c.shape[0]
                        self.class_log_prior[c]=np.log(total_c_docs/total_docs)

                        words_count=x_c.sum(axis=0)+ self.alpha
                        total_words=words_count.sum()

                        self.feature_log_prior[c]=np.log(words_count/total_words)
        def predict(self,x):
                results=[]
                for i in range(x.shape[0]):
                        sample=x[i]
                        class_score={}
                        for c in self.classes:
                                log_prior=self.class_log_prior[c]
                                log_liklihood=sample@ self.feature_log_prior[c].T
                                score=log_prior+log_liklihood
                                class_score[c]=score
                        predict_class=max(class_score,key=class_score.get)
                        results.append(predict_class)
                return np.array(results)

model=mulitnomialNB()

model.fit(x_v_train,y_train)

y_pred=model.predict(x_v_test)

print(accuracy_score(y_test,y_pred))

print(confusion_matrix(y_test,y_pred))


joblib.dump(vectorizer, "vectorizer.pkl")
joblib.dump(model, "model.pkl")


print("Model and vectorizer saved!")





0.9901345291479821
[[968   2]
 [  9 136]]
Model and vectorizer saved!
